In [11]:
# Importar os pacotes

# lib para http
import requests

# Lib para minerar os dados da pagina
from bs4 import BeautifulSoup

# Lib para modelagem de dados
import pandas as pd

# Lib para análise gráfica
import plotly.graph_objects as Dash

In [94]:
# Carregar a pagina

# URL para buscar os dados
Site_Base = 'https://books.toscrape.com/catalogue/page-1.html'

# Carregar os dados da pagina
Pagina = requests.get( Site_Base )

In [23]:
# 200 - ok
Pagina

<Response [200]>

In [24]:
# Verificando tipo
type( Pagina )

requests.models.Response

In [25]:
# BS4 vai intrepretar o arquivo html
Coleta = BeautifulSoup(Pagina.text, 'html.parser')

In [26]:
# Tipo
type( Coleta )

bs4.BeautifulSoup

In [27]:
# Coletando o titulo da pagina
Coleta.title

<title>
    All products | Books to Scrape - Sandbox
</title>

In [107]:
article_produto = Coleta.find_all('article', attrs={'class' : 'product_pod'})

# Vericando
print(f'Foi localizado { len(article_produto) } articles')

Foi localizado 20 articles


In [103]:
# Exemplo para buscar
classe_p = Coleta.find_all('p')[0]
review = classe_p.get('class')[1]
review

'Five'

In [139]:
article_produto[2].find_all('p')[0]

<p class="star-rating One">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>

In [147]:

# Listas dos dados dos livros
nome_livros = []
nota_livro = []
preco_livros = []

# Loop para correr as paginas
for pagina in range(1, 6):
  Site = f'https://books.toscrape.com/catalogue/page-{pagina}.html'
  Pagina = requests.get( Site )
  Coleta = BeautifulSoup(Pagina.text, 'html.parser')

  # Localizando os articles pela classe
  article_produto = Coleta.find_all('article', attrs={'class' : 'product_pod'})

  # Loop para correr pelos artigos buscando as informações
  for posicao, consulta in enumerate(article_produto):

    # busca o nome do livro
    tag_a = article_produto[posicao].find_all('h3')[0].find_all('a')[0]
    nome = tag_a.get('title')

    # busca a nota do livro
    tag_p = article_produto[posicao].find_all('p')[0]
    review = tag_p.get('class')[1]
    if review == 'One':
      review = 1
    elif review == 'Two':
      review = 2
    elif review == 'Three':
      review = 3
    elif review == 'Four':
      review = 4
    else:
      review = 5

    # busca o preço do livro
    preco = article_produto[posicao].find_all('p')[1].text[2:]


    # Salva a informação nas listas
    nome_livros.append( nome )

    nota_livro.append( review )

    preco_livros.append( preco )

    #print( posicao, nome, preco, review )

# Organizando os dados
Dicionario = {
    'Nome do Livro' : nome_livros,
    'Nota do Livro' : nota_livro,
    'Preço do Livro' : preco_livros
}

# Criar a tabela
Tabela_Web = pd.DataFrame( Dicionario )

#Verificar
Tabela_Web.head()


,Nome do Livro,Nota do Livro,Preço do Livro
0,A Light in the Attic,3,51.77
1,Tipping the Velvet,1,53.74
2,Soumission,1,50.10
3,Sharp Objects,4,47.82
4,Sapiens: A Brief History of Humankind,5,54.23


In [148]:
Tabela_Web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome do Livro   100 non-null    object
 1   Nota do Livro   100 non-null    int64 
 2   Preço do Livro  100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB
